In [1]:
setwd('/Users/alexis/Library/CloudStorage/OneDrive-UniversityofNorthCarolinaatChapelHill/CEMALB_DataAnalysisPM/Projects/P1001. InVitroWildfire EVProteomics/P1001.3. Analyses/P1001.3.5. Group Distribution Analysis/Input')
Output = ('/Users/alexis/Library/CloudStorage/OneDrive-UniversityofNorthCarolinaatChapelHill/CEMALB_DataAnalysisPM/Projects/P1001. InVitroWildfire EVProteomics/P1001.3. Analyses/P1001.3.5. Group Distribution Analysis/Output')
cur_date = "062722"

library(readxl)
library(tidyverse)
library(reshape2)


#reading in file
proteomics_df = data.frame(read_excel("062122_Imputed_Proteomics_Data.xlsx"))
protein_info_df = data.frame(read_excel("List of Proteins Detected_041922.xlsx")) %>%
    rename(Protein = Accession)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.5     ✔ purrr   0.3.4
✔ tibble  3.1.6     ✔ dplyr   1.0.8
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


Attaching package: ‘reshape2’


The following object is masked from ‘package:tidyr’:

    smiths




In [2]:
head(proteomics_df)
head(protein_info_df)

,Set,Treatment,ID,Protein,Value
,<chr>,<chr>,<chr>,<chr>,<dbl>
1,Set1,CoCulture_Exposed,CoCulture_Exposed1,P35555,6540459348
2,Set1,CoCulture_Exposed,CoCulture_Exposed1,ENSEMBL.ENSBTAP00000007350,8526739787
3,Set1,CoCulture_Exposed,CoCulture_Exposed1,Q15149,3467760325
4,Set1,CoCulture_Exposed,CoCulture_Exposed1,Q3SZ57,10425228827
5,Set1,CoCulture_Exposed,CoCulture_Exposed1,ENSEMBL.ENSBTAP00000024146,11522621335
6,Set1,CoCulture_Exposed,CoCulture_Exposed1,O15230,2820298745


,Protein,Gene.name,Description,GOCC.name,Contaminant
,<chr>,<chr>,<chr>,<chr>,<chr>
1,P02751,FN1,Fibronectin OS=Homo sapiens OX=9606 GN=FN1 PE=1 SV=5,apical plasma membrane;basal lamina;blood microparticle;cell part;cytoplasmic membrane-bounded vesicle lumen;cytoplasmic part;cytoplasmic vesicle part;endoplasmic reticulum-Golgi intermediate compartment;extracellular matrix;extracellular matrix part;extracellular membrane-bounded organelle;extracellular organelle;extracellular region;extracellular region part;extracellular space;extracellular vesicular exosome;fibrinogen complex;intracellular membrane-bounded organelle;intracellular organelle;intracellular organelle part;intracellular part;macromolecular complex;membrane part;membrane-bounded organelle;membrane-bounded vesicle;membrane-enclosed lumen;organelle;organelle lumen;organelle part;plasma membrane part;platelet alpha granule lumen;protein complex;secretory granule lumen;vesicle;vesicle lumen,-
2,P02769,SWISS-PROT:P02769 (Bos taurus) Bovine serum albumin precursor,SWISS-PROT:P02769 (Bos taurus) Bovine serum albumin precursor,NA,+
3,P07996,THBS1,Thrombospondin-1 OS=Homo sapiens OX=9606 GN=THBS1 PE=1 SV=2,cell part;cell surface;cytoplasmic membrane-bounded vesicle;cytoplasmic membrane-bounded vesicle lumen;cytoplasmic part;cytoplasmic vesicle;cytoplasmic vesicle part;endoplasmic reticulum;endoplasmic reticulum lumen;endoplasmic reticulum part;external side of plasma membrane;extracellular matrix;extracellular membrane-bounded organelle;extracellular organelle;extracellular region;extracellular region part;extracellular space;extracellular vesicular exosome;fibrinogen complex;intracellular membrane-bounded organelle;intracellular organelle;intracellular organelle lumen;intracellular organelle part;intracellular part;macromolecular complex;membrane part;membrane-bounded organelle;membrane-bounded vesicle;membrane-enclosed lumen;organelle;organelle lumen;organelle part;plasma membrane part;platelet alpha granule;platelet alpha granule lumen;protein complex;sarcoplasmic reticulum;secretory granule lumen;stored secretory granule;vesicle;vesicle lumen,-
4,Q2UVX4,SWISS-PROT:Q2UVX4 (Bos taurus) Complement C3 precursor,SWISS-PROT:Q2UVX4 (Bos taurus) Complement C3 precursor,NA,+
5,P98160,HSPG2,Basement membrane-specific heparan sulfate proteoglycan core protein OS=Homo sapiens OX=9606 GN=HSPG2 PE=1 SV=4,adherens junction;anchoring junction;basal lamina;cell junction;cell part;cell-substrate adherens junction;cell-substrate junction;cytoplasmic part;extracellular matrix;extracellular matrix part;extracellular membrane-bounded organelle;extracellular organelle;extracellular region;extracellular region part;extracellular space;extracellular vesicular exosome;focal adhesion;Golgi apparatus part;Golgi lumen;intracellular organelle lumen;intracellular organelle part;intracellular part;lysosomal lumen;membrane;membrane-bounded organelle;membrane-bounded vesicle;membrane-enclosed lumen;organelle;organelle lumen;organelle part;plasma membrane;vacuolar lumen;vacuolar part;vesicle,-
6,P12763,SWISS-PROT:P12763 (Bos taurus) Alpha-2-HS-glycoprotein precursor,SWISS-PROT:P12763 (Bos taurus) Alpha-2-HS-glycoprotein precursor,NA,+


In [3]:
proteomics_df = proteomics_df %>%
    # just want a column that specifies the treatment and exposure separately
    separate(Treatment, c("Treatment", "Exposure"), sep = "_") 

head(proteomics_df)

,Set,Treatment,Exposure,ID,Protein,Value
,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>
1,Set1,CoCulture,Exposed,CoCulture_Exposed1,P35555,6540459348
2,Set1,CoCulture,Exposed,CoCulture_Exposed1,ENSEMBL.ENSBTAP00000007350,8526739787
3,Set1,CoCulture,Exposed,CoCulture_Exposed1,Q15149,3467760325
4,Set1,CoCulture,Exposed,CoCulture_Exposed1,Q3SZ57,10425228827
5,Set1,CoCulture,Exposed,CoCulture_Exposed1,ENSEMBL.ENSBTAP00000024146,11522621335
6,Set1,CoCulture,Exposed,CoCulture_Exposed1,O15230,2820298745


In [4]:
# splitting by Set
proteomics_df_split = proteomics_df %>%
    group_by(Set) %>%
    group_split()

set1_proteomics_df = proteomics_df_split[[1]]
set2_proteomics_df = proteomics_df_split[[2]]

head(set1_proteomics_df)

Set,Treatment,Exposure,ID,Protein,Value
<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>
Set1,CoCulture,Exposed,CoCulture_Exposed1,P35555,6540459348
Set1,CoCulture,Exposed,CoCulture_Exposed1,ENSEMBL.ENSBTAP00000007350,8526739787
Set1,CoCulture,Exposed,CoCulture_Exposed1,Q15149,3467760325
Set1,CoCulture,Exposed,CoCulture_Exposed1,Q3SZ57,10425228827
Set1,CoCulture,Exposed,CoCulture_Exposed1,ENSEMBL.ENSBTAP00000024146,11522621335
Set1,CoCulture,Exposed,CoCulture_Exposed1,O15230,2820298745


Running t tests to determine if there are statistically significant distribution differences in co-culture unexposed vs. exposed and tri-culture unexposed vs. exposed. 

In [5]:
t_test_values = function(df, set_id){
    # """
    # Running t tests after filtering for set, treatment, exposure, and protein using a loop. 
    # Ultimately using this test to compare proteins (unexposed vs. exposed).

    # :param: subsetted dataframe, empty dataframe
    # :output: a dataframe containing the set, treatment, exposure, protein, u stat, p value, p adj

    # """
    treatments = unique(df$Treatment)
    proteins = unique(df$Protein)
    
    values_df = data.frame()
    # iterating through each tx, protein

    for(i in 1:length(treatments)){
        for(j in 1:length(proteins)){
            # unexposed df
            unexposed_df = df %>%
                filter(Treatment == treatments[i], Exposure == "Vehicle", Protein == proteins[j])
            # exposed df
            exposed_df = df %>%
                filter(Treatment == treatments[i], Exposure == "Exposed", Protein == proteins[j])

            # t test
            t_test = t.test(unexposed_df$Value, exposed_df$Value)
            
            # calculating FC to get directionality
            FC = log2(mean(unexposed_df$Value)/mean(exposed_df$Value))

            # contains smoking status compared, compartment, cytokine, u stat, and p value
            values_vector = cbind(set_id, treatments[i], proteins[j], FC, t_test$statistic, t_test$p.value)
            values_df = rbind(values_df, values_vector)
            }
        }

    
    # adding col names
    colnames(values_df) = c("Set", "Treatment", "Protein", "log2FC", "Statistic", "PValue")
    
    
    # calculating padj values
    padj_vector = c()
    # adjusting within each treatment
    for (i in 1:length(treatments)){
        PAdj = p.adjust(as.numeric(as.character(values_df$PValue)), method = "fdr")
        padj_vector = c(padj_vector, PAdj)
    }
    
    values_df = cbind(values_df, padj_vector) %>%
        rename(PAdj = padj_vector)

    values_df$log2FC = as.numeric(values_df$log2FC) 
    values_df$Statistic = as.numeric(values_df$Statistic)
    values_df$PValue = as.numeric(values_df$PValue) 
    
    return(values_df)
}

In [ ]:
# calling fn
set1_t_test = t_test_values(set1_proteomics_df, "Set1")
set2_t_test = t_test_values(set2_proteomics_df, "Set2")

In [ ]:
# creating 1 df
t_test_df = rbind(set1_t_test, set2_t_test)

# adding in gene names
final_t_test_df = inner_join(t_test_df, protein_info_df %>%
                                select(c("Protein", "Gene.name")))
final_t_test_df = final_t_test_df[, c(1,2,3,8,4,5,6,7)]
head(final_t_test_df)

In [ ]:
# exporting t test results
write.csv(final_t_test_df, paste0(Output,"/", cur_date, "_Statistical_Results_Only.csv"), row.names = FALSE)

In [ ]:
t_test_df = pivot_longer(t_test_df, cols = 4:7, names_to = "var", values_to = "Value") %>%
    unite("Variable", Treatment, var, sep = ".") %>%
    select(-Set)

t_test_df = t_test_df[,c(2,1,3)]

head(t_test_df)

In [ ]:
#calculating overall avg. abundance and within smoking groups
avg_abundance_df = proteomics_df %>%
    group_by(Protein) %>%
    summarize(NA.Avg_Abundance = mean(Value))

#stratified by treatment
avg_abundance_df_stratified = proteomics_df %>%
    group_by(Treatment, Protein) %>%
    summarize(Avg_Abundance_stratified = mean(Value))

head(avg_abundance_df)

In [ ]:
#changing the stratified dfs to making combining easier
#first splitting dfs
split_avg_abundance_stratified = avg_abundance_df_stratified %>%
    group_by(Treatment) %>%
    group_split()

#adding treatment to the title
coculture_exposed_avg_abundance = split_avg_abundance_stratified[[1]] %>%
    select(-Treatment) %>%
    rename(Coculture_Exposed.Avg_Abundance = Avg_Abundance_stratified)
coculture_vehicle_avg_abundance = split_avg_abundance_stratified[[2]] %>%
    select(-Treatment) %>%
    rename(Coculture_Vehicle.Avg_Abundance = Avg_Abundance_stratified)
triculture_exposed_avg_abundance = split_avg_abundance_stratified[[1]] %>%
    select(-Treatment) %>%
    rename(Triculture_Exposed.Avg_Abundance = Avg_Abundance_stratified)
triculture_vehicle_avg_abundance = split_avg_abundance_stratified[[2]] %>%
    select(-Treatment) %>%
    rename(Triculture_Vehicle.Avg_Abundance = Avg_Abundance_stratified)

head(coculture_exposed_avg_abundance)

In [ ]:
# combining into 1 abundance df
stratified_abundance_df = inner_join(inner_join(coculture_exposed_avg_abundance, coculture_vehicle_avg_abundance), 
      inner_join(triculture_exposed_avg_abundance, triculture_vehicle_avg_abundance))
abundance_df = inner_join(avg_abundance_df, stratified_abundance_df)

head(abundance_df)

In [ ]:
# reformatting the t test df
t_test_df %>%
    # in order to separate using periods it must be specified as "\\."
    separate(Variable, c("Treatment", "Variable"), sep = "\\.") %>%
    # expanding the variable col
    pivot_wider(names_from = "Variable", values_from = "Value")

In [32]:
# # melting df so combining with other data frames is easier/ more clear
# abundance_df = melt(abundance_df, variable.name = "Variable", value.name = "Value")
# head(abundance_df)

Using Protein as id variables



,Protein,Variable,Value
,<chr>,<fct>,<dbl>
1,A1L4H1,NA.Avg_Abundance,120967180
2,ENSEMBL.ENSBTAP00000006074,NA.Avg_Abundance,592914915
3,ENSEMBL.ENSBTAP00000007350,NA.Avg_Abundance,6573830475
4,ENSEMBL.ENSBTAP00000011227,NA.Avg_Abundance,45175441
5,ENSEMBL.ENSBTAP00000013050,NA.Avg_Abundance,340900160
6,ENSEMBL.ENSBTAP00000014147,NA.Avg_Abundance,165427820


In [45]:
# combining into 1 df
# statistical_df = rbind(t_test_df, abundance_df) 
# combined_df = left_join(statistical_df, protein_info_df)
# combined_df[1:2,]

Joining, by = "Protein"


Protein,Variable,Value,Gene.name,Description,GOCC.name,Contaminant
<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>
P35555,CoCulture.log2FC,0.4170353,FBN1,Fibrillin-1 OS=Homo sapiens OX=9606 GN=FBN1 PE=1 SV=4,basement membrane;extracellular matrix;extracellular matrix part;extracellular membrane-bounded organelle;extracellular organelle;extracellular region;extracellular region part;extracellular space;extracellular vesicular exosome;fibril;membrane-bounded organelle;membrane-bounded vesicle;microfibril;organelle;proteinaceous extracellular matrix;vesicle,-
P35555,CoCulture.Statistic,1.6983966,FBN1,Fibrillin-1 OS=Homo sapiens OX=9606 GN=FBN1 PE=1 SV=4,basement membrane;extracellular matrix;extracellular matrix part;extracellular membrane-bounded organelle;extracellular organelle;extracellular region;extracellular region part;extracellular space;extracellular vesicular exosome;fibril;membrane-bounded organelle;membrane-bounded vesicle;microfibril;organelle;proteinaceous extracellular matrix;vesicle,-


In [15]:
# exporting
write.csv(final_df, paste0(Output,"/", cur_date, "_Distribution_Analysis_All_Data.csv"), row.names = FALSE)